In [3]:
import pickle
import multiprocessing

from time import time

import gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser

print(" setup complete")

 setup complete


## Load Data

In [4]:
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

## Variables

In [5]:
maxLen = 150
BATCH_SIZE = 32 #20
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 200

## Bigrams

In [6]:
sent = []
for col in ['question', 'question_body', 'answer']:
    words = [row.split() for row in X_train[col]]
    sent = sent + words

In [7]:
sentences = Phrases(sent)

In [8]:
bigram = Phraser(sentences)

In [9]:
sentences = bigram[sent]

## Gensim word to vector

In [10]:
cores = multiprocessing.cpu_count() 
cores

8

In [11]:
len(sentences)

30000

In [12]:
fast_text_model = FastText(min_count=50,
                     window=10,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=50,
                     workers=cores-1)

In [13]:
#size = 100 
#window = 10
epochs = 50

In [14]:
t = time() 
fast_text_model.build_vocab(sentences, progress_per=10000)
print('time to build vocabulary : ', format(round((time() - t) / 60, 2)))

time to build vocabulary :  14.53


## Training Model

In [15]:
t = time()
fast_text_model.train(sentences, total_examples=fast_text_model.corpus_count, epochs=50, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 11.97 mins


In [16]:
pretrained_weights = fast_text_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

/Users/axelchenu/Work_directory/DATA_SCIENCE_YNOV/SMART_FAQ/smartFAQ/envs/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [17]:
with open('../../data/fast_text_model_pretrained_weights.pickle', 'wb') as handle:
    pickle.dump(fast_text_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
fast_text_model.most_similar(['python'])

/Users/axelchenu/Work_directory/DATA_SCIENCE_YNOV/SMART_FAQ/smartFAQ/envs/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[("'python", 0.7595863938331604),
 ('bin_python', 0.6522622108459473),
 ('Python', 0.623629629611969),
 ('ipython', 0.6014732122421265),
 ('python_script', 0.5755845904350281),
 ('python_interpreter', 0.5676314234733582),
 ('python_exe', 0.5632917881011963),
 ('python_org', 0.5140905380249023),
 ('python_site', 0.5070676207542419),
 ('pythonic', 0.4951178729534149)]

In [ ]:
# © Axel CHENU 2021

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e80043e2-6875-4b65-a196-a0ffb97a1282' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>